In [1]:
!pip install flask

In [2]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import mysql.connector
import jwt, datetime
from werkzeug.security import generate_password_hash, check_password_hash

app = Flask(__name__)
CORS(app)
app.config['SECRET_KEY'] = 'your-secret-key'

# Database connection
def get_connection():
    return mysql.connector.connect(
        host="localhost",
        port=3308,
        user="root",
        password="",
        database="lost_found_db"
    )

# Test connection
try:
    conn = get_connection()
    print("Connected to lost_found_db successfully!")
    conn.close()
except Exception as e:
    print("Connection failed:", e)


Connected to lost_found_db successfully!


In [3]:
# Register Route
@app.route('/api/register', methods=['POST'])
def register():
    data = request.get_json()
    name = data.get('name')
    email = data.get('email')
    phone = data.get('phone')
    password = data.get('password')
    role = 'seeker'  # Set role here

    try:
        conn = get_connection()
        cursor = conn.cursor()
        cursor.execute(
            "INSERT INTO users (name, email, phone_number, password, role) VALUES (%s, %s, %s, %s, %s)",
            (name, email, phone, password, role)
        )
        conn.commit()
        return jsonify({'message': 'User registered successfully'}), 201
    except mysql.connector.Error as err:
        return jsonify({'error': str(err)}), 400
    finally:
        cursor.close()
        conn.close()


In [4]:
@app.route('/api/login', methods=['POST'])
def login():
    data = request.get_json()
    email = data.get('email')
    password = data.get('password')

    conn = get_connection()
    cursor = conn.cursor(dictionary=True)
    try:
        cursor.execute("SELECT * FROM users WHERE email = %s AND password = %s", (email, password))
        user = cursor.fetchone()

        if user:
            # 🔐 Generate token
            token = jwt.encode({
                'user_id': user['id'],
                'role': user['role'],
                'exp': datetime.datetime.utcnow() + datetime.timedelta(hours=1)
            }, 'your_secret_key', algorithm='HS256')

            # ✅ Return user info and token
            return jsonify({
                'message': 'Login successful',
                'user': {
                    'id': user['id'],
                    'email': user['email'],
                    'role': user['role'],
                    'name': user['name']
                },
                'token': token
            })
        else:
            return jsonify({'message': 'Invalid email or password'}), 401
    finally:
        cursor.close()
        conn.close()

In [ ]:
if __name__ == '__main__':
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [02/Aug/2025 22:40:46] "GET /api/search?query=jhjj HTTP/1.1" 404 -
127.0.0.1 - - [02/Aug/2025 22:44:59] "GET /api/search?query=A%20K%20Perera HTTP/1.1" 404 -
127.0.0.1 - - [02/Aug/2025 22:53:21] "GET /api/search?query=U%20P%20Silva HTTP/1.1" 404 -
